In [165]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from IPython import display
import pylab as pl

import nipype.pipeline.engine as pe        
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber

import sys
from medpy.io import load
import numpy as np
from sklearn.feature_extraction import image
from scipy import ndimage
import math
from inspect import signature
from IPython import display
import pylab as pl

import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, upsample_2d

from traits.api import *

import nibabel as nib

from nipype.utils.filemanip import split_filename
from nipype.interfaces.base import traits, isdefined, CommandLine, CommandLineInputSpec,\
    TraitedSpec, File, InputMultiPath, OutputMultiPath, BaseInterface, BaseInterfaceInputSpec
from nipype.interfaces.mixins import reporting

In [181]:
import subprocess
def run(self, command, env={}, cwd=os.getcwd()):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.run(command, shell=True,
                               env=merged_env, cwd=cwd, capture_output=True)
    return process

In [217]:
class BrainExtractionInputSpec(BaseInterfaceInputSpec):
    base_dir = Directory(desc='Root directory',mandatory=True,exists=True)
    in_file = File(desc='Input image',mandatory=True)
    in_ckpt = File(desc='Network_checkpoint',mandatory=True)
    threshold = traits.Float(0.5,desc='Threshold determining cutoff probability (0.5 by default)')
    out_postfix = traits.Str("_masked.nii.gz", usedefault=True)
    #out_file = File(mandatory=True, desc= 'Output image')

class BrainExtractionOutputSpec(TraitedSpec):
    out_file = File(desc='Brain masked image')

class BrainExtraction(BaseInterface):

    input_spec = BrainExtractionInputSpec
    output_spec = BrainExtractionOutputSpec
    
    def _run_interface(self, runtime): 
        
        cmd = 'python brainExtraction.py -i "{}" -c "{}" -t "{}" -o "{}" '.format(self.inputs.in_file,self.inputs.in_ckpt,
                                                                         self.inputs.threshold,self.inputs.out_postfix)
        #extractBrain(self.inputs.in_file,self.inputs.in_ckpt,self.inputs.threshold,self.inputs.out_postfix)
        try:
            print('... cmd: {}'.format(cmd))
            run(self, cmd, env={}, cwd=os.path.abspath(self.inputs.base_dir))
        except:
            print('Failed')    
        return runtime

    def _list_outputs(self):

        return {'out_file': self.inputs.in_file[:-4]+self.inputs.out_postfix}
    
    
class MultipleBrainExtractionInputSpec(BaseInterfaceInputSpec):
    base_dir = Directory(desc='Root directory',mandatory=True,exists=True)
    input_images = InputMultiPath(File(desc='MRI Images', mandatory = True))
    in_ckpt = File(desc='Network_checkpoint',mandatory=True)
    threshold = traits.Float(0.5,desc='Threshold determining cutoff probability (0.5 by default)')
    out_postfix = traits.Str("_masked.nii.gz", usedefault=True)
    
class MultipleBrainExtractionOutputSpec(TraitedSpec):
    output_images = OutputMultiPath(File())

class MultipleBrainExtraction(BaseInterface):
    input_spec = MultipleBrainExtractionInputSpec
    output_spec = MultipleBrainExtractionOutputSpec

    def _run_interface(self, runtime):
        #if len(self.inputs.input_images)>0:
        for input_image in self.inputs.input_images:
            ax = BrainExtraction(base_dir = self.inputs.base_dir, in_file = input_image, in_ckpt= self.inputs.in_ckpt,threshold = self.inputs.threshold, out_postfix=self.inputs.out_postfix)
            ax.run()
        return runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['output_images'] = glob(os.path.abspath("*.nii.gz"))
        return outputs    

In [218]:
test_dir = "/home/hkebiri/Desktop/dti_ANDRAS_few_examples_with_t2/export_normal/original-t2/NIFTI_HK/TestBrainExtraction"
modelCkpt = '/media/hkebiri/Storage/4Unet/model/GPU/2019-09-07_18:21:22_1/Unet.ckpt-88000'
wf = Workflow(name="BrainExtractWorfklow", base_dir=test_dir)

#Data grabber node 
dg = Node(interface=DataGrabber(outfields = ['mriImg']), name='data_grabber')
dg.inputs.base_directory = test_dir
dg.inputs.template = '*'
dg.inputs.raise_on_empty = True             
dg.inputs.field_template = dict(mriImg=os.path.join('6','*.nii'))
dg.inputs.sort_filelist = False

#Brain mask node
#brainMask = Node(interface = BrainExtraction(),name='Brain_extraction')
brainMask = Node(interface = MultipleBrainExtraction(),name='Multiple_Brain_extraction') 

brainMask.inputs.base_dir = test_dir
brainMask.inputs.threshold = 0.5
brainMask.inputs.in_ckpt = modelCkpt

#Connecting nodes
wf.connect(dg, "mriImg", brainMask, "input_images")

res = wf.run()

200528-05:27:30,968 nipype.workflow INFO:
	 Workflow BrainExtractWorfklow settings: ['check', 'execution', 'logging', 'monitoring']
200528-05:27:30,974 nipype.workflow INFO:
	 Running serially.
200528-05:27:30,975 nipype.workflow INFO:
	 [Node] Setting-up "BrainExtractWorfklow.data_grabber" in "/home/hkebiri/Desktop/dti_ANDRAS_few_examples_with_t2/export_normal/original-t2/NIFTI_HK/TestBrainExtraction/BrainExtractWorfklow/data_grabber".
200528-05:27:30,979 nipype.workflow INFO:
	 [Node] Running "data_grabber" ("nipype.interfaces.io.DataGrabber")
200528-05:27:30,985 nipype.workflow INFO:
	 [Node] Finished "BrainExtractWorfklow.data_grabber".
200528-05:27:30,987 nipype.workflow INFO:
	 [Node] Setting-up "BrainExtractWorfklow.Multiple_Brain_extraction" in "/home/hkebiri/Desktop/dti_ANDRAS_few_examples_with_t2/export_normal/original-t2/NIFTI_HK/TestBrainExtraction/BrainExtractWorfklow/Multiple_Brain_extraction".
200528-05:27:30,992 nipype.workflow INFO:
	 [Node] Running "Multiple_Brain_ext